In [5]:
import pandas as pd
from datatable import dt, f

Após a coleta e agregação dos dados, para facilitar a leitura e organização dos dados para realização do trabalho, com o entendimento sobre as principais características e atributos da base que nos seriam interessantes, fizemos os seguintes passos de filtragem:

### Seleção dos Cursos _STEM_

Para atender o escopo da pesquisa em relação ao STEM, foram selecionados cursos dessas áreas observando os dados do dataset, para encontrar:

- 51: ciências biológicas e correlatas

- 54: matemática e estatística

- 53: ciências físicas

- 61: computação

- 71: engenharias e correlatas

### Colunas

Alguns nomes das colunas eram muito extensos e outros eram abreviados. Com isso, para um melhor entendimento do que se tratava cada coluna, resolvemos fazer as seguintes renomeações e filtrar as colunas que nos seriam de interesse com base no dicionário de dados.

In [6]:
columns = {
  'NU_ANO_CENSO': 'ano',                 #  Ano do Censo
  'NO_REGIAO': 'regiao',                 #  Nome da região geográfica do local de oferta do curso
  'SG_UF': 'uf',                         #  Sigla da Unidade da Federação do local de oferta do curso
  'CO_MUNICIPIO': 'cod_municipio',       #  Código do IBGE do municipio do local de oferta do curso
  'NO_MUNICIPIO': 'nome_municipio',      #  Nome do Município do local de oferta do curso
  'IN_CAPITAL': 'capital',               #  Informa se o local de oferta do curso está localizado em capital de Estado
  'CO_CINE_AREA_ESPECIFICA': 'cod_area', # Código de identificação da área específica, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco
  'NO_CINE_AREA_ESPECIFICA': 'area',     #  Nome da área específica, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco
  'NO_CINE_ROTULO': 'curso',             #  Nome do curso, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco
  'TP_REDE': 'rede_publica',             #  Rede de Ensino
  'TP_MODALIDADE_ENSINO': 'presencial',  #  Tipo da modalidade de ensino do curso
  'TP_GRAU_ACADEMICO': 'bacharel',       #  Tipo do grau acadêmico conferido ao ao aluno pela conclusão dos requisitos exigidos pelo curso
  'QT_ING': 'ingressantes',              #  Quantidade de ingressante no curso
  'QT_ING_FEM': 'ingressantes_fem',      #  Quantidade de ingressante do sexo feminino no curso
  'QT_CONC': 'concluintes',              #  Quantidade de concluinte no curso
  'QT_CONC_FEM': 'concluintes_fem',      #  Quantidade de concluinte no curso do sexo feminino
  ...: None,
}

Tendo isso, construímos a função de filtragem, para extrair as áreas e colunas de maior interesse da pesquisa.

In [7]:
def filter_dataset(dataset):
  df = dataset[
    (
      (f.cod_area == 51)
      |
      (f.cod_area == 54)
      |
      (f.cod_area == 53)
      |
      (f.cod_area == 61)
      |
      (f.cod_area == 61)
      |
      (f.cod_area == 71)
    ) & (
      (f.ingressantes > 0)
      &
      (f.concluintes > 0)
      &
      (f.concluintes_fem > 1)
      &
      (f.ingressantes > f.concluintes)
      &
      (f.ingressantes_fem > f.concluintes_fem)
    )
    , :]
  return df

### Aplicando o tratamento
Após isso, manipulações adicionais serão feitas para tentar alcançar maior facilidade na leitura.

In [33]:
# redefinindo get_path_table_by_year
def get_path_table_by_year(year):
  path_table = '/'.join([
    '../data/raw',
    'censo_{year}.zip',
    'Microdados do Censo da Educaç╞o Superior {year}',
    'dados',
    'MICRODADOS_CADASTRO_CURSOS_{year}.CSV'
  ]).format(year = year)
  return path_table

In [35]:
def read_datasets(from_year, to_year, columns):
  dfs = list()
  # fazendo a agregação levando em conta os novos nomes das colunas
  for y in range(from_year, to_year + 1):
    path_table = get_path_table_by_year(y)
    dataset = dt.fread(path_table, columns=columns)
    df_filter = filter_dataset(dataset)
    dfs.append(df_filter)
  df =  dt.rbind(*dfs)
  df = df.sort(f.ano, f.cod_municipio, f.area, f.curso, -f.concluintes_fem)
  df = df.to_pandas()
  df['rede_publica'] = df['rede_publica'].map({1: True, 2: False})
  df['presencial'] = df['presencial'].map({1: True, 2: False})
  df['bacharel'] = df['bacharel'].map({1: True, 3: False})
  df['frac_ingressantes'] = df['ingressantes_fem'] / df['ingressantes']
  df['frac_concluintes'] = df['concluintes_fem'] / df['concluintes']
  df.dropna(inplace=True)
  return df

In [36]:
from_year = 2010
to_year = 2019
df = read_datasets(from_year, to_year, columns)
df = df[list(columns.values())[:-1] + ['frac_ingressantes', 'frac_concluintes']]
df

,ano,regiao,uf,cod_municipio,nome_municipio,capital,cod_area,area,curso,rede_publica,presencial,bacharel,ingressantes,ingressantes_fem,concluintes,concluintes_fem,frac_ingressantes,frac_concluintes
0,2010,Norte,RO,1100049.0,Cacoal,False,51,Ciências biológicas e correlatas,Biologia,False,True,True,21,13,6,4,0.619048,0.666667
1,2010,Norte,RO,1100049.0,Cacoal,False,71,Engenharia e profissões correlatas,Gestão ambiental,False,True,False,61,26,27,12,0.426230,0.444444
2,2010,Norte,RO,1100049.0,Cacoal,False,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,88,48,10,5,0.545455,0.500000
3,2010,Norte,RO,1100064.0,Colorado do Oeste,False,71,Engenharia e profissões correlatas,Gestão ambiental,True,True,False,59,33,23,11,0.559322,0.478261
4,2010,Norte,RO,1100122.0,Ji-Paraná,False,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,109,52,32,15,0.477064,0.468750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17989,2019,Centro-Oeste,DF,5300108.0,Brasília,True,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,25,13,6,3,0.520000,0.500000
17990,2019,Centro-Oeste,DF,5300108.0,Brasília,True,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,17,7,9,3,0.411765,0.333333
17991,2019,Centro-Oeste,DF,5300108.0,Brasília,True,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,24,8,3,2,0.333333,0.666667
17992,2019,Centro-Oeste,DF,5300108.0,Brasília,True,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,48,14,20,2,0.291667,0.100000


In [37]:
output_file = '../data/cursos.csv.zip'
df.to_csv(output_file, index=False)

In [38]:
df_read = pd.read_csv(
  output_file, dtype={'cod_municipio': int})
df_read

,ano,regiao,uf,cod_municipio,nome_municipio,capital,cod_area,area,curso,rede_publica,presencial,bacharel,ingressantes,ingressantes_fem,concluintes,concluintes_fem,frac_ingressantes,frac_concluintes
0,2010,Norte,RO,1100049,Cacoal,False,51,Ciências biológicas e correlatas,Biologia,False,True,True,21,13,6,4,0.619048,0.666667
1,2010,Norte,RO,1100049,Cacoal,False,71,Engenharia e profissões correlatas,Gestão ambiental,False,True,False,61,26,27,12,0.426230,0.444444
2,2010,Norte,RO,1100049,Cacoal,False,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,88,48,10,5,0.545455,0.500000
3,2010,Norte,RO,1100064,Colorado do Oeste,False,71,Engenharia e profissões correlatas,Gestão ambiental,True,True,False,59,33,23,11,0.559322,0.478261
4,2010,Norte,RO,1100122,Ji-Paraná,False,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,109,52,32,15,0.477064,0.468750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17974,2019,Centro-Oeste,DF,5300108,Brasília,True,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,25,13,6,3,0.520000,0.500000
17975,2019,Centro-Oeste,DF,5300108,Brasília,True,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,17,7,9,3,0.411765,0.333333
17976,2019,Centro-Oeste,DF,5300108,Brasília,True,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,24,8,3,2,0.333333,0.666667
17977,2019,Centro-Oeste,DF,5300108,Brasília,True,71,Engenharia e profissões correlatas,Gestão ambiental,False,False,False,48,14,20,2,0.291667,0.100000


In [39]:
df_read.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17979 entries, 0 to 17978
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ano                17979 non-null  int64  
 1   regiao             17979 non-null  object 
 2   uf                 17979 non-null  object 
 3   cod_municipio      17979 non-null  int64  
 4   nome_municipio     17979 non-null  object 
 5   capital            17979 non-null  bool   
 6   cod_area           17979 non-null  int64  
 7   area               17979 non-null  object 
 8   curso              17979 non-null  object 
 9   rede_publica       17979 non-null  bool   
 10  presencial         17979 non-null  bool   
 11  bacharel           17979 non-null  bool   
 12  ingressantes       17979 non-null  int64  
 13  ingressantes_fem   17979 non-null  int64  
 14  concluintes        17979 non-null  int64  
 15  concluintes_fem    17979 non-null  int64  
 16  frac_ingressantes  179